In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import json
import codecs

In [2]:
from scraperutils import *

In [3]:
url = "https://www.ussoccer.com/schedule-tickets"
html = urlopen(url)

In [4]:
soup = BeautifulSoup(html)

In [5]:
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=2)[0].findAll('th')]

In [6]:
column_headers

['Date', 'Time', 'Matchup', 'Venue', 'Watch']

In [7]:
data_rows = soup.findAll('tr')[1:]

In [8]:
data_rows

[<tr class="year-2019 team-U-20-WNT no-links" itemscope="" itemtype="https://schema.org/SportsEvent">
 <td><time datetime="2019-03-03T06:00:00Z" itemprop="startDate">March 3, 2019</time></td>
 <td>6 AM ET</td>
 <td>
 <meta content="U-20 WNT vs France U-19" itemprop="name"/>
 <a href="/us-under20-womens-national-team/tournaments/2019-womens-u19-la-manga-tournament/20190303-u20wnt-v-france-u19-la-manga" itemprop="url">
 <span itemprop="competitor" itemscope="" itemtype="https://schema.org/SportsTeam"><span itemprop="name">U-20 WNT</span></span>
 <text>vs</text>
 <span itemprop="competitor" itemscope="" itemtype="https://schema.org/SportsTeam"><span itemprop="name">France U-19</span></span>
 </a>
 </td>
 <td itemprop="location" itemscope="" itemtype="https://schema.org/Place"><span itemprop="address"><meta content="La Manga Club; Cartagena, Spain" itemprop="name"/>La Manga Club; Cartagena, Spain</span></td>
 <td itemprop="offers" itemscope="" itemtype="https://schema.org/Offer">
 <br/>
 <

In [9]:
player_data = [[cleanString(td.getText()) for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [10]:
df = pd.DataFrame(player_data, columns=column_headers)


In [11]:
df.head()

,Date,Time,Matchup,Venue,Watch
0,"March 3, 2019",6 AM ET,U-20 WNT vs France U-19,"La Manga Club; Cartagena, Spain",
1,"March 5, 2019",8 AM ET,U-20 WNT vs Sweden U-19,"La Manga Club; Cartagena, Spain",
2,"March 5, 2019",8 PM ET,WNT vs Brazil - SheBelieves Cup,"Raymond James Stadium, Tampa, Fla.",FS1
3,"March 21, 2019",8 PM ET,MNT vs Ecuador,"Orlando City Stadium; Orlando, Fla.","ESPN2, UniMas, UDN"
4,"March 24, 2019",2:30 PM ET,U-23 WNT vs Reign FC,"Merlo Field, University of Portland; Portland,...",


In [12]:
df["Venue"]

0                       La Manga Club; Cartagena, Spain
1                       La Manga Club; Cartagena, Spain
2                    Raymond James Stadium, Tampa, Fla.
3                   Orlando City Stadium; Orlando, Fla.
4     Merlo Field, University of Portland; Portland,...
5                  BBVA Compass Stadium; Houston, Texas
6     Merlo Field, University of Portland; Portland,...
7     Merlo Field, University of Portland; Portland,...
8      Dick’s Sporting Goods Park, Commerce City, Colo.
9       Banc of California Stadium; Los Angeles, Calif.
10                         IMG Academy; Bradenton, Fla.
11                         IMG Academy; Bradenton, Fla.
12                         IMG Academy; Bradenton, Fla.
13                  Levi’s Stadium; Santa Clara, Calif.
14                        Busch Stadium; St. Louis, Mo.
15         Bielsko-Biala Stadium; Bielsko-Biala, Poland
16                       Red Bull Arena; Harrison, N.J.
17         Bielsko-Biala Stadium; Bielsko-Biala,

In [13]:
# df["Date"] = pd.to_datetime(df["Date"])

In [13]:
df['Matchup'] = df['Matchup'].apply(replaceSponsor)

In [14]:
df['Matchup']

0             U-20 WNT vs France U-19
1             U-20 WNT vs Sweden U-19
2     WNT vs Brazil - SheBelieves Cup
3                      MNT vs Ecuador
4                U-23 WNT vs Reign FC
5                        MNT vs Chile
6      U-23 WNT vs Portland Thorns FC
7       U-23 WNT vs Chicago Red Stars
8                    WNT vs Australia
9                      WNT vs Belgium
10                 U-17 MNT vs Canada
11               U-17 MNT vs Barbados
12              U-17 MNT vs Guatemala
13                WNT vs South Africa
14                 WNT vs New Zealand
15    U-20 MNT vs Ukraine - World Cup
16                      WNT vs Mexico
17    U-20 MNT vs Nigeria - World Cup
18      U-20 MNT vs Qatar - World Cup
19        WNT vs Thailand - World Cup
20           WNT vs Chile - World Cup
21              MNT vs TBD - Gold Cup
22          WNT vs Sweden - World Cup
23              MNT vs TBD - Gold Cup
24              MNT vs TBD - Gold Cup
Name: Matchup, dtype: object

In [16]:
df["Venue"] = df["Venue"].apply(cleanString)

In [17]:
df["OpponentTeam"] = df["Matchup"].apply(getOpponentTeam)

In [18]:
df["USTeam"] = df["Matchup"].apply(getUSTeam)

In [19]:
df["Competition"] = df["Matchup"].apply(getCompetition)

In [20]:
df.drop('Matchup', axis='columns', inplace=True)

In [21]:
df["Venue"]

0                       La Manga Club; Cartagena, Spain
1                       La Manga Club; Cartagena, Spain
2                    Raymond James Stadium, Tampa, Fla.
3                   Orlando City Stadium; Orlando, Fla.
4     Merlo Field, University of Portland; Portland,...
5                  BBVA Compass Stadium; Houston, Texas
6     Merlo Field, University of Portland; Portland,...
7     Merlo Field, University of Portland; Portland,...
8      Dick’s Sporting Goods Park, Commerce City, Colo.
9       Banc of California Stadium; Los Angeles, Calif.
10                         IMG Academy; Bradenton, Fla.
11                         IMG Academy; Bradenton, Fla.
12                         IMG Academy; Bradenton, Fla.
13                  Levi’s Stadium; Santa Clara, Calif.
14                        Busch Stadium; St. Louis, Mo.
15         Bielsko-Biala Stadium; Bielsko-Biala, Poland
16                       Red Bull Arena; Harrison, N.J.
17         Bielsko-Biala Stadium; Bielsko-Biala,

In [23]:
df.head()

,Date,Time,Venue,Watch,OpponentTeam,USTeam,Competition
0,"March 3, 2019",6 AM ET,"La Manga Club; Cartagena, Spain",,France U-19,U-20 WNT,
1,"March 5, 2019",8 AM ET,"La Manga Club; Cartagena, Spain",,Sweden U-19,U-20 WNT,
2,"March 5, 2019",8 PM ET,"Raymond James Stadium, Tampa, Fla.",FS1,Brazil,WNT,SheBelieves Cup
3,"March 21, 2019",8 PM ET,"Orlando City Stadium; Orlando, Fla.","ESPN2, UniMas, UDN",Ecuador,MNT,
4,"March 24, 2019",2:30 PM ET,"Merlo Field, University of Portland; Portland,...",,Reign FC,U-23 WNT,


In [31]:
data = json.dumps(json.loads(df.to_json(orient='records')), indent=2)

In [33]:
d = json.loads(data)

In [35]:
with open('schedule.json', 'w') as outfile:
    json.dump(d,outfile,ensure_ascii=False)